In [37]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl

from pandas_datareader import data as pdr

import datetime as dt
import yfinance as yf
import holidays

from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
import plotly.graph_objects as go
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

In [38]:
#### Functions Required

### Step 1 - extract_all and multiindex_to_wide functions

## For Step 1.1: downloading of stock prices. only 1 stock
def extract_all(stocks: list, 
                start: dt.datetime, 
                end: dt.datetime):
            
    def extract_single_ticker(ticker):
           
        return(yf
               .download(ticker, 
                               start,
                               end)
               )
    interim_df = map(extract_single_ticker, stocks) # applies the extract_single_ticker function to the stocks (list of tickers)
    
    return(pd
           .concat(interim_df,
                  keys = stocks,
                  names = ["ticker", "date"]
                 )
          )
    
## For Step 1.2: converting multiindex DF long format to singleindex wide format
def multiindex_to_wide(stock_MI: pd.DataFrame):
    
    stock_wide =\
    (
        stock_MI['Adj Close'].reset_index()\
            .pivot(index = 'date',
                columns = 'ticker',
                values = 'Adj Close')
            
    )
    

In [39]:
### Step 1: Download Data 
tickers = ['IXN','VNQ','XLE','XLF','XLY'] 
start = dt.datetime(2014, 1, 1)  
end = dt.datetime(2024, 10, 6)

data = extract_all(stocks = tickers,
                   start = start,
                   end = end)

data = data.reset_index(level = ['ticker'])

### Group By 'Ticker', resample by 'BMS' to get first observation of each month
data = data.groupby('ticker').resample('BMS').first()
display(data)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


ticker        Open        High         Low       Close  \
ticker date                                                                
IXN    2014-01-01    IXN   13.783333   13.783333   13.683333   13.711667   
       2014-02-03    IXN   13.398333   13.425000   13.096667   13.120000   
       2014-03-03    IXN   13.851667   13.941667   13.825000   13.865000   
       2014-04-01    IXN   14.168333   14.255000   14.168333   14.248333   
       2014-05-01    IXN   14.083333   14.166667   14.041667   14.058333   
...                  ...         ...         ...         ...         ...   
XLY    2024-06-03    XLY  176.089996  176.669998  174.250000  175.889999   
       2024-07-01    XLY  183.110001  183.869995  182.070007  183.020004   
       2024-08-01    XLY  187.570007  188.130005  181.320007  182.889999   
       2024-09-02    XLY  186.580002  187.449997  183.699997  184.550003   
       2024-10-01    XLY  200.250000  200.429993  196.960007  199.559998   

                    Adj Close   Volume  
ticker date                             
IXN    2014-01-01   12.486847  1387200  
       2014-02-03   11.948030   444600  
       2014-03-03   12.626484   491400  
       2014-04-01   12.975574   615600  
       2014-05-01   12.802546   826800  
...                       ...      ...  
XLY    2024-06-03  175.151016  3964500  
       2024-07-01  182.643143  3196600  
       2024-08-01  182.513412  3647900  
       2024-09-02  184.169998  4938200  
       2024-10-01  199.559998  4114500  

[650 rows x 7 columns]

In [40]:
data.loc[data['ticker'].isin(['IXN']), ['ticker','Adj Close', 'Volume']]

ticker  Adj Close   Volume
ticker date                                 
IXN    2014-01-01    IXN  12.486847  1387200
       2014-02-03    IXN  11.948030   444600
       2014-03-03    IXN  12.626484   491400
       2014-04-01    IXN  12.975574   615600
       2014-05-01    IXN  12.802546   826800
...                  ...        ...      ...
       2024-06-03    IXN  76.904732   284200
       2024-07-01    IXN  83.620003   243600
       2024-08-01    IXN  77.680000   843000
       2024-09-02    IXN  77.599998   725700
       2024-10-01    IXN  80.809998   216300

[130 rows x 3 columns]

In [41]:
data.loc[data['ticker'].isin(['IXN']), ['ticker','Adj Close', 'Volume']].droplevel(level = 'ticker')

,ticker,Adj Close,Volume
date,,,
2014-01-01,IXN,12.486847,1387200
2014-02-03,IXN,11.948030,444600
2014-03-03,IXN,12.626484,491400
2014-04-01,IXN,12.975574,615600
2014-05-01,IXN,12.802546,826800
...,...,...,...
2024-06-03,IXN,76.904732,284200
2024-07-01,IXN,83.620003,243600
2024-08-01,IXN,77.680000,843000


In [42]:
### Step 2: Preprocess to be ready to be used for Prophet

dict_of_ticker_dfs = {}

for ticker in data['ticker'].unique():
    ticker_df = data.loc[data['ticker'].isin([ticker]), ['ticker','Adj Close', 'Volume']].droplevel(level = 'ticker')
    ticker_df = ticker_df.reset_index()
    # rename date column to 'ds', forecasted variable to be 'y'
    ticket_df = ticker_df.rename(columns ={'date':'ds', 'Adj Close':'y'})
    ticker_df = ticket_df.loc[:,['ds','y']]
    dict_of_ticker_dfs[ticker] = ticker_df

## 1 example
display(dict_of_ticker_dfs.get('IXN'))

,ds,y
0,2014-01-01,12.486847
1,2014-02-03,11.948030
2,2014-03-03,12.626484
3,2014-04-01,12.975574
4,2014-05-01,12.802546
...,...,...
125,2024-06-03,76.904732
126,2024-07-01,83.620003
127,2024-08-01,77.680000
128,2024-09-02,77.599998


In [43]:
data['ticker'].unique()

array(['IXN', 'VNQ', 'XLE', 'XLF', 'XLY'], dtype=object)

In [44]:
### Step 3 Splitting Data

# Function to split time series data
def split_time_series(df, ticker_name, target_col, feature_cols, n_splits=5, 
                      test_size = 10):
    
    df = dict_of_ticker_dfs.get(ticker_name)
    # Prepare X and y
    X = df[feature_cols]
    y = df[target_col]
    
    # Initialize TimeSeriesSplit
    tss = TimeSeriesSplit(n_splits=n_splits,
                          test_size = test_size)
    
    # Lists to store the indices
    train_indices_list = []
    test_indices_list = []
    
    # Perform the splits
    for train_idx, test_idx in tss.split(X):
        train_indices_list.append(train_idx)
        test_indices_list.append(test_idx)
    
    # Get the last split (usually used for final evaluation)
    last_train_indices = train_indices_list[-1]
    last_test_indices = test_indices_list[-1]
    
    # Split the data
    X_train = X.iloc[last_train_indices]
    X_test = X.iloc[last_test_indices]
    y_train = y.iloc[last_train_indices]
    y_test = y.iloc[last_test_indices]
    
    train = pd.concat(objs = [X_train, y_train], axis = 1)
    test = pd.concat(objs = [X_test, y_test], axis = 1)
    
    train['ticker'] = ticker
    test['ticker'] = ticker
    
    return train, test, tss

train, test, tss = split_time_series(ticker_df, 'IXN', ['y'], ['ds'])

display(test)

,ds,y,ticker
120,2024-01-01,66.176392,XLY
121,2024-02-01,70.647369,XLY
122,2024-03-01,74.639305,XLY
123,2024-04-01,74.898788,XLY
124,2024-05-01,69.689301,XLY
125,2024-06-03,76.904732,XLY
126,2024-07-01,83.620003,XLY
127,2024-08-01,77.680000,XLY
128,2024-09-02,77.599998,XLY
129,2024-10-01,80.809998,XLY


In [45]:
nyse_holidays = holidays.financial_holidays(market ='NYSE',
                                            years=range(2014, 2025))

# Create a dictionary with the dates and names of the holidays
holidays_dict = {date: name for date, name in nyse_holidays.items()}

# Convert to DataFrame
holidays_df = pd.DataFrame(list(holidays_dict.items()), columns=["date", "holiday"])
holidays_df["Date"] = pd.to_datetime(holidays_df["date"], format="%Y-%m-%d")

# Sort the DataFrame by date
holidays_df = holidays_df.sort_values(by="date").reset_index(drop=True)

#rearranging columns so prophet can use 
holidays_df = holidays_df.loc[:,["holiday","date"]]
holidays_df = holidays_df.rename(columns ={'date':'ds'})
display(holidays_df)

,holiday,ds
0,New Year's Day,2014-01-01
1,Martin Luther King Jr. Day,2014-01-20
2,Washington's Birthday,2014-02-17
3,Good Friday,2014-04-18
4,Memorial Day,2014-05-26
...,...,...
97,Juneteenth National Independence Day,2024-06-19
98,Independence Day,2024-07-04
99,Labor Day,2024-09-02
100,Thanksgiving Day,2024-11-28


In [46]:
len(test)

10

#### Fit Prophet

In [47]:
### Step 5 Fit Model

## 5.2 fixed holidays
def fit_model(train, test):
    ticker = train['ticker'].unique().item() 
    model = Prophet(holidays=holidays_df)
    model.add_country_holidays(country_name='US')
    model.fit(train)

    num_periods = len(test)
    future = model.make_future_dataframe(periods = num_periods, freq='M')

    forecast = model.predict(future) 
    print("Confidence Intervals of Predictions")
    fig = plot_plotly(model, forecast)

    fig.add_trace(go.Scatter(
        x=test['ds'],
        y=test['y'],
        mode='markers',
        name='Out-of-Sample Prices',
        marker=dict(color='red')
    ))

    display(fig)

    # Performance Metric
    test_len = len(test)
    train_len = len(train)

    rmse_out_sample = np.sqrt(mean_squared_error(y_true = test["y"], y_pred = forecast['yhat'].iloc[-test_len:]))
    mae_out_sample = mean_absolute_error(y_true = test["y"], y_pred = forecast['yhat'].iloc[-test_len:])
    
    rmse_in_sample = np.sqrt(mean_squared_error(y_true = train["y"], y_pred = forecast['yhat'].iloc[:train_len]))
    mae_in_sample = mean_absolute_error(y_true = train["y"], y_pred = forecast['yhat'].iloc[:train_len])
    
    mape_out_sample = mean_absolute_percentage_error(y_true = test["y"], y_pred = forecast['yhat'].iloc[-test_len:])
    
    print(ticker)
    # print("In Sample Mean Squared Error (MSE):", rmse_in_sample)
    # print("In Sample Mean Absolute Error (MAE):", mae_in_sample)
    print("Out Sample Mean Absolute Percentage Error (MAPE):", mape_out_sample)
    print("Out Sample Mean Squared Error (MSE):", rmse_out_sample)
    print("Out Sample Mean Absolute Error (MAE):", mae_out_sample)
    return (ticker, model, forecast, rmse_out_sample, mae_out_sample, mape_out_sample)
    
(ticker, model, forecast, rmse_out_sample, mae_out_sample, mape_out_sample) = fit_model(train, test)

15:21:55 - cmdstanpy - INFO - Chain [1] start processing
15:21:56 - cmdstanpy - INFO - Chain [1] done processing
c:\Users\limfy\anaconda3\Lib\site-packages\prophet\forecaster.py:1854: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



Confidence Intervals of Predictions


XLY
Out Sample Mean Absolute Percentage Error (MAPE): 0.1379640763637384
Out Sample Mean Squared Error (MSE): 11.467837173771866
Out Sample Mean Absolute Error (MAE): 10.609112509458743


In [48]:
plot_components_plotly(model, forecast)

In [49]:
forecast

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,Christmas Day,Christmas Day_lower,Christmas Day_upper,Christmas Day (observed),...,holidays,holidays_lower,holidays_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2014-01-01,9.956578,4.446393,15.504494,9.956578,9.956578,0.0,0.0,0.0,0.0,...,-7.084429,-7.084429,-7.084429,7.349141,7.349141,7.349141,0.0,0.0,0.0,10.221290
1,2014-02-03,10.279080,4.707870,15.411942,10.279080,10.279080,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.006462,0.006462,0.006462,0.0,0.0,0.0,10.285542
2,2014-03-03,10.552719,5.290800,15.458203,10.552719,10.552719,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,-0.216974,-0.216974,-0.216974,0.0,0.0,0.0,10.335745
3,2014-04-01,10.836130,6.307859,16.838238,10.836130,10.836130,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.311521,0.311521,0.311521,0.0,0.0,0.0,11.147650
4,2014-05-01,11.129313,3.873909,14.388066,11.129313,11.129313,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,-1.981199,-1.981199,-1.981199,0.0,0.0,0.0,9.148114
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,2024-05-31,65.035811,61.100486,71.316484,64.959137,65.100106,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,1.087053,1.087053,1.087053,0.0,0.0,0.0,66.122864
126,2024-06-30,65.546469,58.512971,69.375364,65.447565,65.637979,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,-1.734619,-1.734619,-1.734619,0.0,0.0,0.0,63.811850
127,2024-07-31,66.074150,59.955410,70.080910,65.953322,66.187182,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,-1.245855,-1.245855,-1.245855,0.0,0.0,0.0,64.828295
128,2024-08-31,66.601831,61.512818,71.733482,66.460494,66.743350,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.104559,0.104559,0.104559,0.0,0.0,0.0,66.706390


# 5. Repeat for other tickers

In [50]:
list(dict_of_ticker_dfs.keys()
)

['IXN', 'VNQ', 'XLE', 'XLF', 'XLY']

In [51]:
model_results = []
for ticker in list(dict_of_ticker_dfs.keys()):
    train, test, tss = split_time_series(ticker_df, ticker, ['y'], ['ds'])
    (ticker, model, forecast, rmse_out_sample, mae_out_sample, mape_out_sample) = fit_model(train, test)
    result = {"ticker": ticker,
                           "model": model,
                           "forecast": forecast,
                           "out_sample_mape": mape_out_sample,
                           "out_sample_rmse": rmse_out_sample,
                           "out_sample_mae": mae_out_sample}
    
    
    model_results.append(result)

model_results = pd.DataFrame(model_results)
display(model_results.drop(['model','forecast'],axis = 1))
  

15:21:56 - cmdstanpy - INFO - Chain [1] start processing
15:21:56 - cmdstanpy - INFO - Chain [1] done processing
c:\Users\limfy\anaconda3\Lib\site-packages\prophet\forecaster.py:1854: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



Confidence Intervals of Predictions


15:21:57 - cmdstanpy - INFO - Chain [1] start processing


IXN
Out Sample Mean Absolute Percentage Error (MAPE): 0.1379640763637384
Out Sample Mean Squared Error (MSE): 11.467837173771866
Out Sample Mean Absolute Error (MAE): 10.609112509458743


15:21:57 - cmdstanpy - INFO - Chain [1] done processing
c:\Users\limfy\anaconda3\Lib\site-packages\prophet\forecaster.py:1854: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



...

15:21:58 - cmdstanpy - INFO - Chain [1] start processing
15:21:58 - cmdstanpy - INFO - Chain [1] done processing


XLF
Out Sample Mean Absolute Percentage Error (MAPE): 0.10848482286631515
Out Sample Mean Squared Error (MSE): 5.253919039855026
Out Sample Mean Absolute Error (MAE): 4.6036430944769355
Confidence Intervals of Predictions


c:\Users\limfy\anaconda3\Lib\site-packages\prophet\forecaster.py:1854: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



XLY
Out Sample Mean Absolute Percentage Error (MAPE): 0.07950136440543107
Out Sample Mean Squared Error (MSE): 15.985682569732475
Out Sample Mean Absolute Error (MAE): 14.595410650155396


,ticker,out_sample_mape,out_sample_rmse,out_sample_mae
0,IXN,0.137964,11.467837,10.609113
1,VNQ,0.058426,6.255942,5.167430
2,XLE,0.070681,6.810662,6.207856
3,XLF,0.108485,5.253919,4.603643
4,XLY,0.079501,15.985683,14.595411


In [52]:
type(model_results.loc[model_results["ticker"] == 'IXN', "model"].values.item())

prophet.forecaster.Prophet

# 6. Cross Validation for historical performance

In [ ]:
# from prophet.diagnostics import cross_validation
# from prophet.diagnostics import performance_metrics

# def cross_validation_metrics(model):
#     cutoffs = pd.date_range(start='2014-01-01', end='2023-12-01', freq='BMS')
#     df_cv = cross_validation(model, horizon='30 days', cutoffs=cutoffs)
#     df_perf = performance_metrics(df_cv, rolling_window = 0)
#     return df_cv, df_perf

# for ticker in model_results['ticker']:
#     model = model_results.loc[model_results["ticker"] == ticker, "model"].values.item()
#     print("ticker")
#     cutoffs = pd.date_range(start='2022-01-01', end='2023-10-01', freq='BMS')
#     df_cv = cross_validation(model, horizon='30 days', cutoffs=cutoffs)
#     df_perf = performance_metrics(df_cv, rolling_window = 0)
#     display(df_cv)
#     display(df_perf)
    

ticker


  0%|          | 0/21 [00:00<?, ?it/s]

15:21:58 - cmdstanpy - INFO - Chain [1] start processing
15:21:59 - cmdstanpy - INFO - Chain [1] done processing
15:21:59 - cmdstanpy - INFO - Chain [1] start processing
15:22:00 - cmdstanpy - INFO - Chain [1] done processing
15:22:00 - cmdstanpy - INFO - Chain [1] start processing
15:22:00 - cmdstanpy - INFO - Chain [1] done processing


ValueError: Dataframe has no rows.